In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
file_path = "C:/Users/emred/OneDrive/Masaüstü/Docs/3rd Year/Spring/CENG404 - Special Topics in CENG"

In [3]:
ground_truth_df = pd.read_csv(file_path + '/Doktor Raporları/data' + '/mimic-cxr-2.1.0-test-set-labeled.csv')

In [4]:
ground_truth_df

,study_id,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Airspace Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,58085167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,57798090,NaN,1.0,1.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2,50020371,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52225063,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,59039129,NaN,NaN,1.0,NaN,1.0,-1.0,-1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,53953586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
683,59358922,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
684,53225437,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN
685,56814609,NaN,NaN,1.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN


In [5]:
ground_truth_df.columns

Index(['study_id', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation',
       'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
       'Pleural Other', 'Fracture', 'Support Devices'],
      dtype='object')

In [6]:
ground_truth_df

,study_id,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Airspace Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,58085167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,57798090,NaN,1.0,1.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2,50020371,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52225063,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,59039129,NaN,NaN,1.0,NaN,1.0,-1.0,-1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,53953586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
683,59358922,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
684,53225437,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN
685,56814609,NaN,NaN,1.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN


In [7]:
results_df = pd.read_json(file_path + '/new_gemini_labeler_outputs.json')

labels_df = pd.json_normalize(results_df['labels'])

results_df = pd.concat([results_df.drop(columns=['labels']), labels_df], axis = 1)

In [8]:
results_df

,patient_id,report_name,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,p10032725,s50331901,No,No,No,Undefined,No,Undefined,No,No,No,No,No,No,Yes
1,p10032725,s55504914,No,No,No,Undefined,Maybe,Undefined,Undefined,No,Undefined,Undefined,No,No,Yes
2,p10046166,s57977208,Undefined,Undefined,Undefined,Undefined,Undefined,Undefined,Yes,Undefined,Undefined,Undefined,Undefined,No,Undefined
3,p10183775,s52835225,Undefined,Yes,No,Maybe,Undefined,Yes,Undefined,Undefined,No,No,No,No,Undefined
4,p10268877,s53452091,Undefined,Yes,Undefined,Yes,Undefined,No,Undefined,Maybe,Maybe,Undefined,Maybe,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,p16826047,s59712299,Undefined,Maybe,Maybe,Undefined,No,Undefined,Undefined,Maybe,No,Undefined,Maybe,No,Undefined
681,p18110020,s53957798,Undefined,Maybe,No,Undefined,Undefined,Undefined,No,No,Undefined,Undefined,No,Undefined,Yes
682,p18874374,s58085167,Undefined,Yes,Yes,No,Undefined,Undefined,Yes,Yes,Maybe,Undefined,Yes,Maybe,Undefined
683,p18893199,s58971994,No,Undefined,No,No,No,No,No,No,No,No,No,No,Undefined


In [9]:
results_df = results_df.replace({
        'Yes' : 1,
        'No' : 0,
        'Maybe' : -1,
        'Undefined' : np.nan
    }
)

results_df

C:\Users\emred\AppData\Local\Temp\ipykernel_7944\2444437665.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df = results_df.replace({


,patient_id,report_name,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,p10032725,s50331901,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,p10032725,s55504914,0.0,0.0,0.0,NaN,-1.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0
2,p10046166,s57977208,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN
3,p10183775,s52835225,NaN,1.0,0.0,-1.0,NaN,1.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN
4,p10268877,s53452091,NaN,1.0,NaN,1.0,NaN,0.0,NaN,-1.0,-1.0,NaN,-1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,p16826047,s59712299,NaN,-1.0,-1.0,NaN,0.0,NaN,NaN,-1.0,0.0,NaN,-1.0,0.0,NaN
681,p18110020,s53957798,NaN,-1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,1.0
682,p18874374,s58085167,NaN,1.0,1.0,0.0,NaN,NaN,1.0,1.0,-1.0,NaN,1.0,-1.0,NaN
683,p18893199,s58971994,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [10]:
results_df['patient_id'] = results_df["patient_id"].str.replace("p", "").astype(int)

results_df['report_name'] = results_df['report_name'].str.replace('s', '').str.replace('.txt', '').astype(int)

results_df = results_df.rename(columns = {
    'patient_id' : 'subject_id',
    'report_name' : 'study_id'
})

In [11]:
results_df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10032725,50331901,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,10032725,55504914,0.0,0.0,0.0,NaN,-1.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0
2,10046166,57977208,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN
3,10183775,52835225,NaN,1.0,0.0,-1.0,NaN,1.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN
4,10268877,53452091,NaN,1.0,NaN,1.0,NaN,0.0,NaN,-1.0,-1.0,NaN,-1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,16826047,59712299,NaN,-1.0,-1.0,NaN,0.0,NaN,NaN,-1.0,0.0,NaN,-1.0,0.0,NaN
681,18110020,53957798,NaN,-1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,1.0
682,18874374,58085167,NaN,1.0,1.0,0.0,NaN,NaN,1.0,1.0,-1.0,NaN,1.0,-1.0,NaN
683,18893199,58971994,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [12]:
ground_truth_ids = {id for id in ground_truth_df['study_id']}

results_ids = {id for id in results_df['study_id']}

missing_ids = {id for id in ground_truth_ids if id not in results_ids}

with open("relevant_reports.json", "r") as json_file:
    data = json.load(json_file)

filtered_data = [entry for entry in data if int(entry['study_id'].replace("s", "")) in missing_ids]

output_file = "filtered_data.json"
with open(output_file, "w") as json_file:
    json.dump(filtered_data, json_file, indent=4)

print(f'File {output_file} is saved.')

File filtered_data.json is saved.


In [13]:
len(missing_ids)

2

In [14]:
results_df.columns

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object')

In [15]:
ground_truth_df.columns

Index(['study_id', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation',
       'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
       'Pleural Other', 'Fracture', 'Support Devices'],
      dtype='object')

In [16]:
ground_truth_df = ground_truth_df.drop('No Finding', axis = 1)
results_df = results_df.drop('subject_id', axis = 1)

In [17]:
results_df.columns

Index(['study_id', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices'],
      dtype='object')

In [18]:
ground_truth_df.columns

Index(['study_id', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion',
       'Airspace Opacity', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices'],
      dtype='object')

In [19]:
results_df.columns == ground_truth_df.columns

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False, False,  True])

In [20]:
len(ground_truth_df.columns)

14

In [21]:
len(results_df.columns)

14

In [22]:
results_df.columns

Index(['study_id', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices'],
      dtype='object')

In [23]:
ground_truth_df = ground_truth_df.rename(columns = {'Airspace Opacity' : 'Lung Opacity'})

reordered_columns = [col for col in ground_truth_df.columns]

results_df = results_df[reordered_columns]

reordered_columns

['study_id',
 'Enlarged Cardiomediastinum',
 'Cardiomegaly',
 'Lung Lesion',
 'Lung Opacity',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural Effusion',
 'Pleural Other',
 'Fracture',
 'Support Devices']

In [24]:
(results_df['Enlarged Cardiomediastinum'] == 'Undefined').sum()

0

In [25]:
len(results_df)

685

In [26]:
results_df = results_df.drop_duplicates(subset='study_id', keep='first')
results_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,50331901,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
1,55504914,-1.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0
2,57977208,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,52835225,NaN,1.0,NaN,NaN,-1.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,NaN
4,53452091,NaN,1.0,NaN,-1.0,1.0,NaN,-1.0,NaN,0.0,-1.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59712299,0.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,0.0,0.0,NaN,NaN,NaN
681,53957798,NaN,-1.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
682,58085167,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN,-1.0,-1.0,NaN,NaN,NaN
683,58971994,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [27]:
results_df[results_df['study_id'] == 52188580]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
316,52188580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0


In [28]:
len(results_df)

685

#### Element-wise kıyaslama

In [30]:
results_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,50331901,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
1,55504914,-1.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0
2,57977208,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,52835225,NaN,1.0,NaN,NaN,-1.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,NaN
4,53452091,NaN,1.0,NaN,-1.0,1.0,NaN,-1.0,NaN,0.0,-1.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59712299,0.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,0.0,0.0,NaN,NaN,NaN
681,53957798,NaN,-1.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
682,58085167,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN,-1.0,-1.0,NaN,NaN,NaN
683,58971994,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [31]:
# matched_df = results_df.merge(ground_truth_df, on = ['subject_id', 'study_id'], suffixes = ('_results', '_ground_truth'))
matched_df = results_df.merge(ground_truth_df, on = ['study_id'], suffixes = ('_results', '_ground_truth'))

In [32]:
matched_df.columns

Index(['study_id', 'Enlarged Cardiomediastinum_results',
       'Cardiomegaly_results', 'Lung Lesion_results', 'Lung Opacity_results',
       'Edema_results', 'Consolidation_results', 'Pneumonia_results',
       'Atelectasis_results', 'Pneumothorax_results',
       'Pleural Effusion_results', 'Pleural Other_results', 'Fracture_results',
       'Support Devices_results', 'Enlarged Cardiomediastinum_ground_truth',
       'Cardiomegaly_ground_truth', 'Lung Lesion_ground_truth',
       'Lung Opacity_ground_truth', 'Edema_ground_truth',
       'Consolidation_ground_truth', 'Pneumonia_ground_truth',
       'Atelectasis_ground_truth', 'Pneumothorax_ground_truth',
       'Pleural Effusion_ground_truth', 'Pleural Other_ground_truth',
       'Fracture_ground_truth', 'Support Devices_ground_truth'],
      dtype='object')

In [33]:
matched_df

,study_id,Enlarged Cardiomediastinum_results,Cardiomegaly_results,Lung Lesion_results,Lung Opacity_results,Edema_results,Consolidation_results,Pneumonia_results,Atelectasis_results,Pneumothorax_results,...,Lung Opacity_ground_truth,Edema_ground_truth,Consolidation_ground_truth,Pneumonia_ground_truth,Atelectasis_ground_truth,Pneumothorax_ground_truth,Pleural Effusion_ground_truth,Pleural Other_ground_truth,Fracture_ground_truth,Support Devices_ground_truth
0,50331901,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0
1,55504914,-1.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,57977208,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,52835225,NaN,1.0,NaN,NaN,-1.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,53452091,NaN,1.0,NaN,-1.0,1.0,NaN,-1.0,NaN,0.0,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59712299,0.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,0.0,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
681,53957798,NaN,-1.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
682,58085167,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN,-1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
683,58971994,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

results = []

columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]
total = len(results_df) 

for col in columns_to_compare:
    matches = (merged[f"{col}_results"] == merged[f"{col}_ground_truth"]).sum()
    accuracy = matches / total * 100 
    results.append({'Category': col, 'Total': total, 'Matches': matches, 'Accuracy': f"{accuracy:.2f}%"})

accuracy_df = pd.DataFrame(results)

#### ground_truth_df'teki NaN değerlerden ötürü çok düşük bir accuracy çıktı.

In [36]:
accuracy_df

,Category,Total,Matches,Accuracy
0,Enlarged Cardiomediastinum,685,46,6.72%
1,Cardiomegaly,685,214,31.24%
2,Lung Lesion,685,53,7.74%
3,Lung Opacity,685,168,24.53%
4,Edema,685,235,34.31%
5,Consolidation,685,92,13.43%
6,Pneumonia,685,206,30.07%
7,Atelectasis,685,215,31.39%
8,Pneumothorax,685,229,33.43%
9,Pleural Effusion,685,334,48.76%


In [37]:
# Merge the results and ground truth dataframes
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

# Initialize a list to store results
results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]

# Iterate through each column to calculate metrics
for col in columns_to_compare:
    # Get the predictions and ground truth for the current column
    y_pred = merged[f"{col}_results"]
    y_true = merged[f"{col}_ground_truth"]

    # Filter out rows where either y_pred or y_true is NaN (Undefined)
    valid_indices = ~y_pred.isna() & ~y_true.isna()
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]

    # Total valid rows
    total_valid = len(y_true)

    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0

    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN) for each class
    tp = ((y_pred == 1) & (y_true == 1)).sum()  # True Positives for "Yes"
    fp = ((y_pred == 1) & (y_true != 1)).sum()  # False Positives for "Yes"
    fn = ((y_pred != 1) & (y_true == 1)).sum()  # False Negatives for "Yes"

    # Precision, Recall, and F1-Score for "Yes" class
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

    # Append results for the current column
    results.append({
        'Category': col,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
accuracy_df = pd.DataFrame(results)

# Display the resulting DataFrame
accuracy_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,99,46,46.46%,87.50%,33.33%,48.28%
1,Cardiomegaly,268,214,79.85%,98.47%,86.58%,92.14%
2,Lung Lesion,65,53,81.54%,94.00%,88.68%,91.26%
3,Lung Opacity,180,168,93.33%,98.11%,95.12%,96.59%
4,Edema,276,235,85.14%,91.45%,79.85%,85.26%
5,Consolidation,105,92,87.62%,94.12%,87.27%,90.57%
6,Pneumonia,231,206,89.18%,88.64%,70.91%,78.79%
7,Atelectasis,242,215,88.84%,90.52%,96.95%,93.63%
8,Pneumothorax,238,229,96.22%,91.43%,84.21%,87.67%
9,Pleural Effusion,379,334,88.13%,99.15%,84.98%,91.52%


In [38]:
ground_truth_df.isna().sum()

study_id                        0
Enlarged Cardiomediastinum    573
Cardiomegaly                  407
Lung Lesion                   621
Lung Opacity                  494
Edema                         403
Consolidation                 581
Pneumonia                     453
Atelectasis                   444
Pneumothorax                  448
Pleural Effusion              307
Pleural Other                 660
Fracture                      637
Support Devices               453
dtype: int64

In [39]:
ground_truth_df.isna().sum() / len(ground_truth_df) * 100

study_id                       0.000000
Enlarged Cardiomediastinum    83.406114
Cardiomegaly                  59.243086
Lung Lesion                   90.393013
Lung Opacity                  71.906841
Edema                         58.660844
Consolidation                 84.570597
Pneumonia                     65.938865
Atelectasis                   64.628821
Pneumothorax                  65.211063
Pleural Effusion              44.687045
Pleural Other                 96.069869
Fracture                      92.721980
Support Devices               65.938865
dtype: float64

In [40]:
merged.isna().sum()

study_id                                     0
Enlarged Cardiomediastinum_results         302
Cardiomegaly_results                       192
Lung Lesion_results                        368
Lung Opacity_results                       253
Edema_results                              248
Consolidation_results                      240
Pneumonia_results                          169
Atelectasis_results                        237
Pneumothorax_results                       147
Pleural Effusion_results                    87
Pleural Other_results                      348
Fracture_results                           416
Support Devices_results                    321
Enlarged Cardiomediastinum_ground_truth    571
Cardiomegaly_ground_truth                  406
Lung Lesion_ground_truth                   619
Lung Opacity_ground_truth                  493
Edema_ground_truth                         402
Consolidation_ground_truth                 579
Pneumonia_ground_truth                     452
Atelectasis_g

#### Şimdi ise NaN değerleri 0 ile doldurarak deneyelim.

In [42]:
ground_truth_df = ground_truth_df.fillna(0)

results_df = results_df.fillna(0)

In [43]:
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

results = []

columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]
total = len(results_df) 

for col in columns_to_compare:
    matches = (merged[f"{col}_results"] == merged[f"{col}_ground_truth"]).sum()
    accuracy = matches / total * 100 
    results.append({'Category': col, 'Total': total, 'Matches': matches, 'Accuracy': f"{accuracy:.2f}%"})

accuracy_df = pd.DataFrame(results)

In [44]:
accuracy_df

,Category,Total,Matches,Accuracy
0,Enlarged Cardiomediastinum,685,584,85.26%
1,Cardiomegaly,685,547,79.85%
2,Lung Lesion,685,653,95.33%
3,Lung Opacity,685,521,76.06%
4,Edema,685,603,88.03%
5,Consolidation,685,589,85.99%
6,Pneumonia,685,620,90.51%
7,Atelectasis,685,608,88.76%
8,Pneumothorax,685,672,98.10%
9,Pleural Effusion,685,611,89.20%


#### NaN değerlerin sayısı

In [46]:
(ground_truth_df.drop(['study_id'], axis = 1) == np.nan).sum()

Enlarged Cardiomediastinum    0
Cardiomegaly                  0
Lung Lesion                   0
Lung Opacity                  0
Edema                         0
Consolidation                 0
Pneumonia                     0
Atelectasis                   0
Pneumothorax                  0
Pleural Effusion              0
Pleural Other                 0
Fracture                      0
Support Devices               0
dtype: int64

#### NaN değerlerin yüzdesi

In [48]:
(ground_truth_df.drop(['study_id'], axis = 1) == 'Undefined').sum() / len(ground_truth_df) * 100

Enlarged Cardiomediastinum    0.0
Cardiomegaly                  0.0
Lung Lesion                   0.0
Lung Opacity                  0.0
Edema                         0.0
Consolidation                 0.0
Pneumonia                     0.0
Atelectasis                   0.0
Pneumothorax                  0.0
Pleural Effusion              0.0
Pleural Other                 0.0
Fracture                      0.0
Support Devices               0.0
dtype: float64

In [49]:
(results_df.drop(['study_id'], axis = 1) == 'Undefined').sum()

Enlarged Cardiomediastinum    0
Cardiomegaly                  0
Lung Lesion                   0
Lung Opacity                  0
Edema                         0
Consolidation                 0
Pneumonia                     0
Atelectasis                   0
Pneumothorax                  0
Pleural Effusion              0
Pleural Other                 0
Fracture                      0
Support Devices               0
dtype: int64

In [50]:
(results_df.drop(['study_id'], axis = 1) == 'Undefined').sum() / len(results_df) * 100

Enlarged Cardiomediastinum    0.0
Cardiomegaly                  0.0
Lung Lesion                   0.0
Lung Opacity                  0.0
Edema                         0.0
Consolidation                 0.0
Pneumonia                     0.0
Atelectasis                   0.0
Pneumothorax                  0.0
Pleural Effusion              0.0
Pleural Other                 0.0
Fracture                      0.0
Support Devices               0.0
dtype: float64

In [51]:
# Calculate the number of 'Undefined' values for each column
undefined_counts = (results_df.drop(['study_id'], axis=1) == 'Undefined').sum()

# Calculate the percentage of 'Undefined' values for each column
undefined_percentages = (undefined_counts / len(results_df)) * 100

# Combine the counts and percentages into a new DataFrame
undefined_summary_df = pd.DataFrame({
    'Undefined Count': undefined_counts,
    'Undefined Percentage': undefined_percentages
})

# Display the resulting DataFrame
undefined_summary_df

,Undefined Count,Undefined Percentage
Enlarged Cardiomediastinum,0,0.0
Cardiomegaly,0,0.0
Lung Lesion,0,0.0
Lung Opacity,0,0.0
Edema,0,0.0
Consolidation,0,0.0
Pneumonia,0,0.0
Atelectasis,0,0.0
Pneumothorax,0,0.0
Pleural Effusion,0,0.0


In [52]:
# for index, row in matched_df.iterrows():
#     if row['Fracture_results'] != row['Fracture_ground_truth']:
#         print(f"Subject ID: {row['subject_id']}, Study ID: {row['study_id']}, "
#               f"Fracture_results: {row['Fracture_results']}, "
#               f"Fracture_ground_truth: {row['Fracture_ground_truth']}", end ='\n\n')

In [53]:
# Merge the results and ground truth dataframes
merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))

# Initialize a list to store results
results = []

# Columns to compare (excluding 'study_id')
columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]

# Iterate through each column to calculate metrics
for col in columns_to_compare:
    # Get the predictions and ground truth for the current column
    y_pred = merged[f"{col}_results"]
    y_true = merged[f"{col}_ground_truth"]

    # Filter out rows where either y_pred or y_true is NaN (Undefined)
    valid_indices = ~y_pred.isna() & ~y_true.isna()
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]

    # Total valid rows
    total_valid = len(y_true)

    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0

    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN) for each class
    tp = ((y_pred == 1) & (y_true == 1)).sum()  # True Positives for "Yes"
    fp = ((y_pred == 1) & (y_true != 1)).sum()  # False Positives for "Yes"
    fn = ((y_pred != 1) & (y_true == 1)).sum()  # False Negatives for "Yes"

    # Precision, Recall, and F1-Score for "Yes" class
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

    # Append results for the current column
    results.append({
        'Category': col,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
accuracy_df = pd.DataFrame(results)

# Display the resulting DataFrame
accuracy_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,685,584,85.26%,70.00%,30.43%,42.42%
1,Cardiomegaly,685,547,79.85%,76.33%,85.43%,80.62%
2,Lung Lesion,685,653,95.33%,75.81%,88.68%,81.74%
3,Lung Opacity,685,521,76.06%,52.88%,92.31%,67.24%
4,Edema,685,603,88.03%,82.31%,79.26%,80.75%
5,Consolidation,685,589,85.99%,65.75%,85.71%,74.42%
6,Pneumonia,685,620,90.51%,86.67%,70.91%,78.00%
7,Atelectasis,685,608,88.76%,74.61%,96.95%,84.33%
8,Pneumothorax,685,672,98.10%,86.49%,84.21%,85.33%
9,Pleural Effusion,685,611,89.20%,93.17%,84.98%,88.89%


---
## Sadece 1 ve 0'lar üzerinden değerlendirme
---

In [131]:
# Filter out rows where predictions or ground truth are -1 or NaN
copied_merged = merged.copy()

# Iterate through each category to calculate metrics
results = []
categories = [col.replace("_results", "") for col in merged.columns if "_results" in col]

for category in categories:
    # Filter valid rows for the current category
    y_pred = copied_merged[f"{category}_results"]
    y_true = copied_merged[f"{category}_ground_truth"]
    valid_indices = (y_pred.isin([0, 1])) & (y_true.isin([0, 1]))
    
    y_pred = y_pred[valid_indices]
    y_true = y_true[valid_indices]
    
    # Total valid rows
    total_valid = len(y_true)
    
    # Calculate matches for accuracy
    matches = (y_pred == y_true).sum()
    accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0
    
    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    
    # Precision, Recall, and F1-Score
    precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
    recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    
    # Append results for the current category
    results.append({
        'Category': category,
        'Total Valid': total_valid,
        'Matches': matches,
        'Accuracy': f"{accuracy:.2f}%",
        'Precision': f"{precision:.2f}%",
        'Recall': f"{recall:.2f}%",
        'F1-Score': f"{f1_score:.2f}%"
    })

# Create a DataFrame to display the results
filtered_metrics_df = pd.DataFrame(results)

# Display the resulting DataFrame
filtered_metrics_df

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,593,580,97.81%,77.78%,60.87%,68.29%
1,Cardiomegaly,568,528,92.96%,77.25%,98.47%,86.58%
2,Lung Lesion,663,649,97.89%,77.05%,100.00%,87.04%
3,Lung Opacity,668,521,77.99%,52.88%,95.12%,67.97%
4,Edema,599,580,96.83%,89.17%,94.69%,91.85%
5,Consolidation,592,569,96.11%,68.57%,97.96%,80.67%
6,Pneumonia,541,540,99.82%,97.50%,100.00%,98.73%
7,Atelectasis,634,588,92.74%,80.59%,100.00%,89.25%
8,Pneumothorax,670,664,99.10%,91.43%,91.43%,91.43%
9,Pleural Effusion,610,590,96.72%,93.93%,97.89%,95.87%


In [56]:
len(ground_truth_df)

687

In [57]:
results_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,50331901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,55504914,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,57977208,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52835225,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,53452091,0.0,1.0,0.0,-1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59712299,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
681,53957798,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
682,58085167,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,0.0,0.0
683,58971994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
maybe_counts = (ground_truth_df.drop(['study_id'], axis=1) == -1).sum()

maybe_percentages = (maybe_counts / len(ground_truth_df)) * 100

maybe_summary_df = pd.DataFrame({
    '-1 Count': maybe_counts,
    '-1 Percentage': maybe_percentages
})

maybe_summary_df

,-1 Count,-1 Percentage
Enlarged Cardiomediastinum,38,5.531295
Cardiomegaly,39,5.676856
Lung Lesion,9,1.310044
Lung Opacity,0,0.000000
Edema,41,5.967977
Consolidation,26,3.784571
Pneumonia,89,12.954876
Atelectasis,41,5.967977
Pneumothorax,10,1.455604
Pleural Effusion,23,3.347889


In [59]:
ground_truth_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,58085167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57798090,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,50020371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52225063,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59039129,0.0,1.0,0.0,1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,53953586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
683,59358922,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
684,53225437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
685,56814609,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [60]:
results_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,50331901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,55504914,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,57977208,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52835225,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,53452091,0.0,1.0,0.0,-1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59712299,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
681,53957798,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
682,58085167,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,0.0,0.0
683,58971994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
results_df[results_df['study_id'] == 50964400]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
314,50964400,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,1.0,1.0,0.0,0.0,1.0


In [62]:
len(missing_ids)

2

In [63]:
missing_ids

{54107786, 58592606}

In [64]:
ground_truth_df[ground_truth_df['study_id'] == 58592606]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
389,58592606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [65]:
ground_truth_df[ground_truth_df['study_id'] == 54107786]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
332,54107786,0.0,0.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0


In [66]:
results_df[results_df['study_id'] == 54107786]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices


In [67]:
results_df[results_df['study_id'] == 58592606]

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
